In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings('ignore')

Bu ödevde, ev fiyatları verileriyle çalışmaya devam edeceksiniz. Bu ödevi tamamlamak için, aşağıdaki görevlerin çözümlerini içeren bir Jupyter not defterini link olarak gönderin:

1) Ev Fiyatları verilerini Kaggle'dan indirin.

2) Önceki dersteki modelinizi yeniden uygulayın.

3) Aynı model tanımını kullanarak OLS, Lasso, Ridge ve ElasticNet regresyonlarını deneyin. Bu kez, modelleriniz için en iyi hiperparametre değerlerini seçmek için k-fold çapraz doğrulama yapmanız gerekir. Hangi model en iyisidir? Açıklayın?

In [22]:
ev_fiyatlari = pd.read_csv('data/train.csv')
ev_fiyatlari.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
y = ev_fiyatlari['SalePrice']

X = ev_fiyatlari[['YearBuilt', 'TotRmsAbvGrd', 'FullBath' , '1stFlrSF', 'TotalBsmtSF',
                  'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual', 'YearRemodAdd', 'Fireplaces',
                  'BsmtFinSF1', 'WoodDeckSF','2ndFlrSF', 'OpenPorchSF', 'HalfBath']]

***OLS***

In [35]:
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.786
Method:                 Least Squares   F-statistic:                     336.7
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        15:37:30   Log-Likelihood:                -17409.
No. Observations:                1460   AIC:                         3.485e+04
Df Residuals:                    1443   BIC:                         3.494e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.103e+06   1.33e+05     -8.324      0.000   -1.36e+06   -8.43e+05
YearBuilt      198.9533     53.331      3.731      0.000      94.338     303.568
TotRmsAbvGrd  1678.8447   1097.952      1.529      0.126    -474.908    3832.598
FullBath     -2845.5968   2862.053     -0.994      0.320   -8459.827    2768.633
1stFlrSF        32.0764     20.967      1.530      0.126      -9.053      73.206
TotalBsmtSF     12.4115      4.330      2.866      0.004       3.917      20.906
GarageArea      12.6059     10.154      1.241      0.215      -7.313      32.525
GarageCars     1.03e+04   2994.591      3.440      0.001    4427.061    1.62e+04
GrLivArea       17.7291     20.525      0.864      0.388     -22.533      57.991
OverallQual   1.939e+04   1174.531     16.512      0.000    1.71e+04    2.17e+04
YearRemodAdd   323.2454     62.284      5.190      0.000     201.069     445.422
Fireplaces    7755.1333   1798.470      4.312      0.000    4227.237    1.13e+04
BsmtFinSF1      18.3222      2.603      7.038      0.000      13.216      23.429
WoodDeckSF      31.7522      8.181      3.881      0.000      15.704      47.800
2ndFlrSF        24.5893     20.763      1.184      0.236     -16.140      65.318
OpenPorchSF      6.8139     15.825      0.431      0.667     -24.230      37.857
HalfBath     -1066.9575   2791.269     -0.382      0.702   -6542.336    4408.421
==============================================================================
Omnibus:                      596.896   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            97277.099
Skew:                          -0.790   Prob(JB):                         0.00
Kurtosis:                      42.957   Cond. No.                     5.04e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.04e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

****Islemler Oncesi Train ve Test Gruplarinin Olusturulmasi***

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 500)

lrm = LinearRegression()
lrm.fit(X_train, y_train)

y_egitim_tahmini = lrm.predict(X_train)
y_test_tahmini = lrm.predict(X_test)

In [7]:
print("Eğitim kümesindeki gözlem sayısı : {}".format(X_train.shape[0]))
print("Test kümesindeki gözlem sayısı   : {}".format(X_test.shape[0]))

print("Eğitim kümesindeki R-Kare değeri  : {}".format(lrm.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(lrm.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesindeki gözlem sayısı : 1168
Test kümesindeki gözlem sayısı   : 292
Eğitim kümesindeki R-Kare değeri  : 0.7801794585413929
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : 0.8296187584623316
Ortalama Mutlak Hata (MAE)        : 20673.626321627264
Ortalama Kare Hata (MSE)          : 828761729.8038493
Kök Ortalama Kare Hata (RMSE)     : 28788.222067433224
Ortalama Mutlak Yüzde Hata (MAPE) : 13.042167043498004


In [8]:
x_train = sm.add_constant(X_train)

sonuclar = sm.OLS(y_train, X_train).fit()

sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.961
Method:                 Least Squares   F-statistic:                     1806.
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        14:37:12   Log-Likelihood:                -14006.
No. Observations:                1168   AIC:                         2.804e+04
Df Residuals:                    1152   BIC:                         2.812e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
YearBuilt      -31.9344     56.435     -0.566      0.572    -142.661      78.792
TotRmsAbvGrd   697.0836   1291.052      0.540      0.589   -1835.993    3230.160
FullBath      8077.9526   3146.561      2.567      0.010    1904.320    1.43e+04
1stFlrSF        32.4401     24.517      1.323      0.186     -15.664      80.544
TotalBsmtSF     12.9048      5.041      2.560      0.011       3.014      22.796
GarageArea       8.8850     12.215      0.727      0.467     -15.081      32.851
GarageCars    1.474e+04   3536.615      4.167      0.000    7797.688    2.17e+04
GrLivArea        6.3416     23.930      0.265      0.791     -40.609      53.292
OverallQual   2.436e+04   1289.261     18.896      0.000    2.18e+04    2.69e+04
YearRemodAdd   -19.4665     55.532     -0.351      0.726    -128.421      89.488
Fireplaces    5241.3160   2170.326      2.415      0.016     983.082    9499.550
BsmtFinSF1      19.0558      3.063      6.220      0.000      13.045      25.066
WoodDeckSF      40.0425      9.788      4.091      0.000      20.837      59.248
2ndFlrSF        21.7879     24.245      0.899      0.369     -25.781      69.357
OpenPorchSF     12.3433     18.670      0.661      0.509     -24.288      48.974
HalfBath      6534.1558   3221.423      2.028      0.043     213.642    1.29e+04
==============================================================================
Omnibus:                      385.037   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            44354.315
Skew:                          -0.426   Prob(JB):                         0.00
Kurtosis:                      33.177   Cond. No.                     1.22e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

***Ridge fonksiyonu***

In [9]:
from sklearn.linear_model import Ridge

ridgeregr = Ridge(alpha=10**37) 
ridgeregr.fit(X_train, y_train)

y_egitim_tahmini = ridgeregr.predict(X_train)
y_test_tahmini = ridgeregr.predict(X_test)

print("Eğitim kümesi R-Kare değeri       : {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(ridgeregr.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesi R-Kare değeri       : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.004908518792945626
Ortalama Mutlak Hata (MAE)        : 50966.240124788885
Ortalama Kare Hata (MSE)          : 4888036469.351241
Kök Ortalama Kare Hata (RMSE)     : 69914.49398623465
Ortalama Mutlak Yüzde Hata (MAPE) : 32.62793758351267


***Lasso regresyonu***

In [10]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=10**20.5) 
lassoregr.fit(X_train, y_train)

y_egitim_tahmini = lassoregr.predict(X_train)
y_test_tahmini = lassoregr.predict(X_test)

print("Eğitim kümesi R-Kare değeri       : {}".format(lassoregr.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(lassoregr.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesi R-Kare değeri       : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.004908518792945626
Ortalama Mutlak Hata (MAE)        : 50966.240124788885
Ortalama Kare Hata (MSE)          : 4888036469.351241
Kök Ortalama Kare Hata (RMSE)     : 69914.49398623465
Ortalama Mutlak Yüzde Hata (MAPE) : 32.62793758351267


***ElasticNet regresyonu***

In [11]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet(alpha=10**21, l1_ratio=0.5) 
elasticregr.fit(X_train, y_train)

y_egitim_tahmini = elasticregr.predict(X_train)
y_test_tahmini = elasticregr.predict(X_test)

print("Eğitim kümesi R-Kare değeri       : {}".format(elasticregr.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(elasticregr.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesi R-Kare değeri       : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.004908518792945626
Ortalama Mutlak Hata (MAE)        : 50966.240124788885
Ortalama Kare Hata (MSE)          : 4888036469.351241
Kök Ortalama Kare Hata (RMSE)     : 69914.49398623465
Ortalama Mutlak Yüzde Hata (MAPE) : 32.62793758351267


k-fold 

K-Fold Cross Validation, sınıflandırma modellerinin değerlendirilmesi ve modelin eğitilmesi için veri setini parçalara ayırma yöntemlerinden biridir. Bu yazımızda k-fold cross validation (k sayısı kadar çapraz doğrulama) yöntemini anlatmaya çalışacağım.

Elimizde bin kayıtlık bir veri seti olsun. Biz bu veri setinin bir kısmı ile modelimizi eğitmek, bir kısmı ile eğittiğimiz modelimizin başarısını değerlendirmek istiyoruz. Basit yaklaşım; %75’ini eğitim için, %25’ini de test için ayırmaktır. Ancak burada veri parçalanırken verinin dağılımına bağlı olarak modelin eğitim ve testinde bazı sapmalar (bias)  ve hatalar oluşabilir. İşte k-fold cross validation, veriyi belirlenen bir k sayısına göre eşit parçalara böler, her bir parçanın hem eğitim hem de test için kullanılmasını sağlar, böylelikle dağılım ve parçalanmadan kaynaklanan sapma ve hataları asgariye indirir. Ancak modeli k kadar eğitmek ve test etmek gibi ilave bir veri işleme yük ve zamanı ister. Bu durum eğitim ve testi kısa süren küçük ve orta hacimli veriler için sorun olmasa da büyük hacimli veri setlerinde hesaplama ve zaman yönünden maliyetli olabilir.



In [12]:
from sklearn.model_selection import KFold

In [13]:
kf = KFold(n_splits=2)
kf.get_n_splits(X)

2

In [14]:
print(kf)

KFold(n_splits=2, random_state=None, shuffle=False)


In [17]:
KFold(n_splits=2, random_state=None, shuffle=False)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    

TRAIN: [ 730  731  732  733  734  735  736  737  738  739  740  741  742  743
  744  745  746  747  748  749  750  751  752  753  754  755  756  757
  758  759  760  761  762  763  764  765  766  767  768  769  770  771
  772  773  774  775  776  777  778  779  780  781  782  783  784  785
  786  787  788  789  790  791  792  793  794  795  796  797  798  799
  800  801  802  803  804  805  806  807  808  809  810  811  812  813
  814  815  816  817  818  819  820  821  822  823  824  825  826  827
  828  829  830  831  832  833  834  835  836  837  838  839  840  841
  842  843  844  845  846  847  848  849  850  851  852  853  854  855
  856  857  858  859  860  861  862  863  864  865  866  867  868  869
  870  871  872  873  874  875  876  877  878  879  880  881  882  883
  884  885  886  887  888  889  890  891  892  893  894  895  896  897
  898  899  900  901  902  903  904  905  906  907  908  909  910  911
  912  913  914  915  916  917  918  919  920  921  922  923  924  925

KeyError: '[ 730  731  732  733  734  735  736  737  738  739  740  741  742  743\n  744  745  746  747  748  749  750  751  752  753  754  755  756  757\n  758  759  760  761  762  763  764  765  766  767  768  769  770  771\n  772  773  774  775  776  777  778  779  780  781  782  783  784  785\n  786  787  788  789  790  791  792  793  794  795  796  797  798  799\n  800  801  802  803  804  805  806  807  808  809  810  811  812  813\n  814  815  816  817  818  819  820  821  822  823  824  825  826  827\n  828  829  830  831  832  833  834  835  836  837  838  839  840  841\n  842  843  844  845  846  847  848  849  850  851  852  853  854  855\n  856  857  858  859  860  861  862  863  864  865  866  867  868  869\n  870  871  872  873  874  875  876  877  878  879  880  881  882  883\n  884  885  886  887  888  889  890  891  892  893  894  895  896  897\n  898  899  900  901  902  903  904  905  906  907  908  909  910  911\n  912  913  914  915  916  917  918  919  920  921  9

In [21]:
from sklearn.model_selection import train_test_split



In [24]:
ev_fiyatlari.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
y = ev_fiyatlari['SalePrice']

X = ev_fiyatlari[['YearBuilt', 'TotRmsAbvGrd', 'FullBath' , '1stFlrSF', 'TotalBsmtSF',
                  'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual', 'YearRemodAdd', 'Fireplaces',
                  'BsmtFinSF1', 'WoodDeckSF','2ndFlrSF', 'OpenPorchSF', 'HalfBath']]

In [33]:
SalePrice = ev_fiyatlari['SalePrice'].values

In [34]:
sonuc = pd.DataFrame(data = SalePrice)
print(sonuc)

sonuc2 = pd.DataFrame(data = SalePrice, index = range(9), columns= ["YearBuilt", 'TotRmsAbvGrd', 'FullBath' , '1stFlrSF', 'TotalBsmtSF',
                  'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual', 'YearRemodAdd', 'Fireplaces',
                  'BsmtFinSF1', 'WoodDeckSF','2ndFlrSF', 'OpenPorchSF', 'HalfBath'])
print(sonuc2)
SalePrice = ev_fiyatlari['SalePrice'].values
print(SalePrice)

s = pd.concat([sonuc,sonuc2], axis = 1)
print(s)



           0
0     208500
1     181500
2     223500
3     140000
4     250000
5     143000
6     307000
7     200000
8     129900
9     118000
10    129500
11    345000
12    144000
13    279500
14    157000
15    132000
16    149000
17     90000
18    159000
19    139000
20    325300
21    139400
22    230000
23    129900
24    154000
25    256300
26    134800
27    306000
28    207500
29     68500
...      ...
1430  192140
1431  143750
1432   64500
1433  186500
1434  160000
1435  174000
1436  120500
1437  394617
1438  149700
1439  197000
1440  191000
1441  149300
1442  310000
1443  121000
1444  179600
1445  129000
1446  157900
1447  240000
1448  112000
1449   92000
1450  136000
1451  287090
1452  145000
1453   84500
1454  185000
1455  175000
1456  210000
1457  266500
1458  142125
1459  147500

[1460 rows x 1 columns]


ValueError: Shape of passed values is (1, 1460), indices imply (16, 9)

In [31]:
sonuc = pd.DataFrame(data = SalePrice)
print(sonuc)

sonuc2 = pd.DataFrame(data = SalePrice, index = range(9), columns= ["YearBuilt", 'TotRmsAbvGrd', 'FullBath' , '1stFlrSF', 'TotalBsmtSF',
                  'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual', 'YearRemodAdd', 'Fireplaces',
                  'BsmtFinSF1', 'WoodDeckSF','2ndFlrSF', 'OpenPorchSF', 'HalfBath'])
print(sonuc2)
SalePrice = veriler.iloc[:,-1:].values
print(SalePrice)

s = pd.concat([sonuc,sonuc2], axis = 1)
print(s)



         0
0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      19
19      20
20      21
21      22
22      23
23      24
24      25
25      26
26      27
27      28
28      29
29      30
...    ...
1430  1431
1431  1432
1432  1433
1433  1434
1434  1435
1435  1436
1436  1437
1437  1438
1438  1439
1439  1440
1440  1441
1441  1442
1442  1443
1443  1444
1444  1445
1445  1446
1446  1447
1447  1448
1448  1449
1449  1450
1450  1451
1451  1452
1452  1453
1453  1454
1454  1455
1455  1456
1456  1457
1457  1458
1458  1459
1459  1460

[1460 rows x 1 columns]


ValueError: Shape of passed values is (1, 1460), indices imply (16, 9)

In [23]:
X_train, X_test,y_train,y_test = train_test_split(s,SalePrice,test_size=0.33,random=0)

NameError: name 's' is not defined